In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [3]:
batch_size = 128 # Number of Images to send into the network for each pass 
num_classes = 10 # Total number of classes 
epochs = 20 # Total number of times the dataset has to be sent through the network

In [4]:
# the data, shuffled and split between train and test sets. Need Internet connection 
# when you execute it for the first time in your system

x_train = np.genfromtxt('../data/x_train.csv',delimiter=",")
y_train = np.genfromtxt('../data/y_train.csv',delimiter=",")

x_test = np.genfromtxt('../data/x_test.csv',delimiter=",")
y_test = np.genfromtxt('../data/y_test.csv',delimiter=",")

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)


(17000, 1024) (17000,) (3000, 1024) (3000,)


In [5]:
y_test

array([ 7.,  1.,  8., ...,  0.,  6.,  8.])

In [6]:
# We know that MNIST images are 28 pixels in each dimension.
img_size = 32

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 1

In [7]:
x_train = x_train.reshape(17000, img_size_flat) # Flattening the train dataset
x_test = x_test.reshape(3000, img_size_flat) # Flattening the test dataset
x_train = x_train.astype('float32') # Converting the train data into float
x_test = x_test.astype('float32') # Converting the test data into float
x_train /= 255.0 # MinMax Scaling the train data
x_test /= 255.0 # MinMax scaling the test data
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

17000 train samples
3000 test samples


In [8]:
# convert class vectors to binary class matrices - OneHot Encoding
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)
print(x_test)


[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [9]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(1024,))) # Input Dense layer
model.add(Dropout(0.2)) # Adding Dropout Regularization
model.add(Dense(512, activation='relu')) # Hidden Dense Layer 
model.add(Dropout(0.2)) # Adding Dropout Regularization
model.add(Dense(10, activation='softmax')) # Output layer with Softmax activation function

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 792,586
Trainable params: 792,586
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model 
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
metrics=['accuracy'])

In [12]:
# Training the model
history = model.fit(x_train, y_train_cat,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test_cat))

Train on 17000 samples, validate on 3000 samples
Epoch 1/20
17000/17000 [==============================] - 6s 363us/step - loss: 0.3567 - acc: 0.8822 - val_loss: 0.1045 - val_acc: 0.9707
Epoch 2/20
17000/17000 [==============================] - 4s 231us/step - loss: 0.0980 - acc: 0.9675 - val_loss: 0.0715 - val_acc: 0.9800
Epoch 3/20
17000/17000 [==============================] - 4s 230us/step - loss: 0.0587 - acc: 0.9812 - val_loss: 0.0631 - val_acc: 0.9830
Epoch 4/20
17000/17000 [==============================] - 4s 227us/step - loss: 0.0392 - acc: 0.9871 - val_loss: 0.0848 - val_acc: 0.9780
Epoch 5/20
17000/17000 [==============================] - 4s 234us/step - loss: 0.0297 - acc: 0.9898 - val_loss: 0.1134 - val_acc: 0.9760
Epoch 6/20
17000/17000 [==============================] - 4s 236us/step - loss: 0.0228 - acc: 0.9922 - val_loss: 0.0532 - val_acc: 0.9873
Epoch 7/20
17000/17000 [==============================] - 4s 234us/step - loss: 0.0213 - acc: 0.9939 - val_loss: 0.0700 - v

In [13]:
# Evaluating on testing data 
score = model.evaluate(x_test, y_test_cat, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0972290342728
Test accuracy: 0.986666666826


In [14]:
predictions = model.predict(x_test)

In [15]:
y_pred= np.argmax(predictions, axis=-1)   
print(y_pred)
print(y_test)

[7 1 8 ..., 0 6 8]
[ 7.  1.  8. ...,  0.  6.  8.]


In [16]:
print("EVALUATION ON TESTING DATA")
print(classification_report(y_test,y_pred))

EVALUATION ON TESTING DATA
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00       300
        1.0       1.00      0.99      0.99       300
        2.0       0.99      0.95      0.97       300
        3.0       0.95      0.99      0.97       300
        4.0       0.99      0.98      0.99       300
        5.0       0.98      0.98      0.98       300
        6.0       0.98      1.00      0.99       300
        7.0       0.99      0.99      0.99       300
        8.0       0.99      1.00      1.00       300
        9.0       1.00      0.99      0.99       300

avg / total       0.99      0.99      0.99      3000



In [17]:
print ("Confusion matrix")
print(confusion_matrix(y_test,y_pred))

Confusion matrix
[[300   0   0   0   0   0   0   0   0   0]
 [  0 297   0   0   0   0   2   0   1   0]
 [  0   0 286  11   1   1   1   0   0   0]
 [  0   0   2 297   0   1   0   0   0   0]
 [  0   0   0   0 295   2   2   1   0   0]
 [  0   0   1   4   0 293   2   0   0   0]
 [  0   0   0   0   0   0 299   1   0   0]
 [  1   0   0   0   1   1   0 297   0   0]
 [  0   0   0   0   0   0   0   0 300   0]
 [  0   0   1   2   0   0   0   0   1 296]]
